#### Importing Data

In [24]:
import numpy as np
from sklearn.datasets import load_iris

In [25]:
dataset = load_iris(['data', 'target'])

In [26]:
X = dataset[0]
Y = dataset[1]

In [27]:
Y = Y.reshape(150, 1)

#### Creating train and test sets

In [28]:
from sklearn.model_selection import train_test_split

In [29]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, random_state = 1)

#### Scaling Features

In [30]:
from sklearn.preprocessing import StandardScaler

In [31]:
sc = StandardScaler()

In [32]:
X_train = sc.fit_transform(X_train)

In [33]:
X_test = sc.transform(X_test)

#### Using Builtin Model

In [23]:
from sklearn.linear_model import LogisticRegression

In [36]:
logreg = LogisticRegression()
logreg.fit(X_train, Y_train)

C:\Users\shz204\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\utils\validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [37]:
pred = logreg.predict(X_test)

In [40]:
score = logreg.score(X_test, Y_test)
print(score)

0.8157894736842105


#### Transposing

In [41]:
X_train = X_train.T
X_test = X_test.T
Y_train = Y_train.T
Y_test = Y_test.T

#### Model Building

#### Helper Functions

In [50]:
def sigmoid(Z):
    A = 1.0/(1.0 + np.exp(-Z))
    assert(A.shape == Z.shape)
    return A

In [51]:
def relu(Z):
    A = np.maximun(0, Z)
    assert(A.shape == Z.shape)
    return A

#### Initialize Parameters

In [44]:
def initialize_parameters(dim):
    w = np.random.randn(dim, 1)
    b = np.random.randn()
    
    assert(w.shape == (dim, 1))
    assert(isinstance(b, float) or isinstance(b, int))
    
    return w, b

#### Forward and Backward Propagation

In [45]:
def propagate(w, b, X, Y):
    m = X.shape[1]
    
    Z = np.dot(w.T, X) + b
    A = sigmoid(Z)
    cost = (-1/float(m)) * (np.dot(Y, np.log(A).T) + np.dot((1-Y), np.log(1 - A).T))
    
    dw = (1/float(m)) * np.dot(X, (A - Y).T)
    db = (1/float(m)) * np.sum(A - Y)
    
    assert(dw.shape == w.shape)
    assert(db.dtype == float)
    cost = np.squeeze(cost)
    assert(cost.shape == ())
    
    grads = {"dw": dw,
             "db": db}
    
    return grads, cost

#### Optimization

In [46]:
def optimize(w, b, X, Y, num_iterations, learning_rate, print_cost = False):
    costs = []
    
    for i in range(num_iterations):
        grads, cost = propagate(w, b, X, Y)
        
        dw = grads["dw"]
        db = grads["db"]
        
        w = w - learning_rate * dw
        b = b - learning_rate * db
        
        if(i%100 == 0):
            costs.append(cost)
            
        if(print_cost and i%100 == 0):
            print("Cost after iteration %i: %f" %(i, cost))
            
        
    params = {"w": w,
              "b": b}
    
    grads = {"dw": dw,
             "db": db} 
    
    return params, grads, cost

#### Prediction

In [47]:
def predict(w, b, X):
    m = X.shape[1]
    Y_prediction = np.zeros((1, m))
    w = w.reshape(X.shape[0], 1)
    
    Z = np.dot(w.T, X) + b
    A = sigmoid(Z)
    
    for i in range(A.shape[1]):
        Y_prediction[0, i] = 1 if A[0, i] > 0.5 else 0
        
    assert(Y_prediction.shape == (1, m))
    
    return Y_prediction

#### Model

In [58]:
def model(X_train, Y_train, X_test, Y_test, num_iterations = 2000, learning_rate = 0.05, print_cost = False):
    dim = X_train.shape[0]
    w, b = initialize_parameters(dim)
    
    parameters, grads, costs = optimize(w, b, X_train, Y_train, num_iterations, learning_rate, print_cost)
    
    w = parameters["w"]
    b = parameters["b"]
    
    Y_prediction_test = predict(w, b, X_test)
    Y_prediction_train = predict(w, b, X_train)
    
    print("train accuracy: {} %".format(100 - np.mean(np.abs(Y_prediction_train - Y_train)) * 100))
    print("test accuracy: {} %".format(100 - np.mean(np.abs(Y_prediction_test - Y_test)) * 100))

    
    d = {"costs": costs,
         "Y_prediction_test": Y_prediction_test, 
         "Y_prediction_train" : Y_prediction_train, 
         "w" : w, 
         "b" : b,
         "learning_rate" : learning_rate,
         "num_iterations": num_iterations}
    
    return d

In [56]:
d = model(X_train, Y_train, X_test, Y_test, num_iterations = 1000, learning_rate = 0.005, print_cost = True)

Cost after iteration 0: 1.006175
Cost after iteration 100: 0.038502
Cost after iteration 200: -0.541395
Cost after iteration 300: -0.949595
Cost after iteration 400: -1.283141
Cost after iteration 500: -1.581011
Cost after iteration 600: -1.860000
Cost after iteration 700: -2.128206
Cost after iteration 800: -2.389929
Cost after iteration 900: -2.647615
train accuracy: 63.392857142857146 %
test accuracy: 76.31578947368422 %
